In [ ]:
import torch as th
import torch.nn as nn

import random
import numpy as np
# scipy 科学计算包
import scipy.sparse as sp 
from scipy.linalg import fractional_matrix_power, inv
# fractional_matrix_power: Compute the fractional power of a matrix.

!pip install dgl-cu111 dglgo -f https://data.dgl.ai/wheels/repo.html

import dgl 
from dgl.data import CoraGraphDataset, CiteseerGraphDataset, PubmedGraphDataset
from dgl.nn import APPNPConv, GraphConv, AvgPooling
# https://docs.dgl.ai/generated/dgl.nn.pytorch.conv.APPNPConv.html

import networkx as nx

from sklearn.preprocessing import MinMaxScaler


Looking in links: https://data.dgl.ai/wheels/repo.html


In [ ]:
!nvcc -V
!python -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
Python 3.7.12


$S^{PPR} = \alpha{(} I_n - (1-\alpha) {D^{-\frac{1}{2}}} A {D^{-\frac{1}{2}} )}^{-1}$

In [ ]:
def compute_ppr(graph: nx.Graph, alpha = 0.2, self_loop=True):
    adj = nx.convert_matrix.to_numpy_array(graph) # Returns the graph adjacency matrix as a NumPy array.
    if self_loop:
        adj = adj + np.eye(adj.shape[0])
            
    deg = np.diag(np.sum(adj, 1))
    deg_inv = fractional_matrix_power(deg, -0.5)
    adj_norm = np.matmul(np.matmul(deg_inv, adj), deg_inv)
    return alpha * inv(np.eye(adj.shape[0]) - (1 - alpha) * adj_norm)

In [ ]:
# dataset

def process_dataset(name):
    if name == 'cora':
        dataset = CoraGraphDataset()
    else:
        # TODO: other datasets
        pass    
        
    graph = dataset[0]    
    feat = graph.ndata.pop('feat')
    label = graph.ndata.pop('label')
    
    train_mask = graph.ndata.pop('train_mask')
    val_mask = graph.ndata.pop('val_mask')
    test_mask = graph.ndata.pop('test_mask')
    
    train_idx = th.nonzero(train_mask).squeeze()    
    # 等效：train_nids = graph.nodes()[train_mask]
    val_idx = th.nonzero(val_mask).squeeze()
    test_idx = th.nonzero(test_mask).squeeze()
    
    # https://docs.dgl.ai/generated/dgl.add_self_loop.html
    graph = graph.remove_self_loop()
    graph = graph.add_self_loop()
    
    nx_g = dgl.to_networkx(graph)
    
    # diffusion matrices
    diff_adj = compute_ppr(nx_g, 0.2, self_loop=False)
    
    diff_edges = np.nonzero(diff_adj) # ( array( ... ), array( ... ))
    diff_weight = th.Tensor(diff_adj[diff_edges])
    diff_graph = dgl.graph(diff_edges)

    return graph, diff_graph, feat, label, train_idx, val_idx, test_idx, diff_weight

In [ ]:
# process_dataset('cora')

In [ ]:
# model
    
class Discriminator(nn.Module):
    def __init__(self, dim):
        super(Discriminator, self).__init__()
        self.fn = nn.Bilinear(dim, dim, 1)
        
    def forward(self, h1, h2, h3, h4, c1, c2):
        """
        Mutual Information is modeled as discriminator 
        that takes in a node representation from one view and a graph representation from another view, 
        and scores the agreement between them.

        Args:
            h1 : view1 node representation (joint)
            h2 : view2 node representation (joint)
            h3 : view1 node representation (marginal)
            h4 : view2 node representation (marginal)
            c1 : view1 graph representation (joint)
            c2 : view2 graph representation (joint)
        """
        c_x1 = c1.expand_as(h1).contiguous()
        c_x2 = c2.expand_as(h2).contiguous()
        
        # positive
        sc_1 = self.fn(h2, c_x1).squeeze(1)
        sc_2 = self.fn(h1, c_x2).squeeze(1)
        
         # negative
        sc_3 = self.fn(h4, c_x1).squeeze(1)
        sc_4 = self.fn(h3, c_x2).squeeze(1)

        logits = th.cat((sc_1, sc_2, sc_3, sc_4))
        return logits
        
class MVGRL(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(MVGRL, self).__init__()
        # https://docs.dgl.ai/generated/dgl.nn.pytorch.conv.GraphConv.html
        # σ(A ̃XΘ) learn node representation for view1, A ̃ is symmetrically normalized adjacency matrix
        self.encoder1 = GraphConv(in_dim, out_dim, norm='both', bias=True, activation=nn.PReLU())
        # σ(SXΘ) learn node representation for view2, S is diffusion matrix
        self.encoder2 = GraphConv(in_dim, out_dim, norm='none', bias=True, activation=nn.PReLU())
        
        # before the projection head, into a graph representation using a graph pooling (readout) function P
        self.pooling = AvgPooling()
        
        self.disc = Discriminator(out_dim)
        self.act_fn = nn.Sigmoid()
        
    def get_embedding(self, graph, diff_graph, feat, edge_weight):
        h1 = self.encoder1(graph, feat)
        h2 = self.encoder2(diff_graph, feat, edge_weight=edge_weight)
        
        return (h1 + h2).detach()
    
    def forward(self, graph, diff_graph, feat, shuf_feat, edge_weight):
        h1 = self.encoder1(graph, feat)
        h2 = self.encoder2(diff_graph, feat, edge_weight=edge_weight)
        
        h3 = self.encoder1(graph, shuf_feat)
        h4 = self.encoder2(diff_graph, shuf_feat, edge_weight=edge_weight)
        
        c1 = self.act_fn(self.pooling(graph, h1))
        c2 = self.act_fn(self.pooling(graph, h2))
        
        out = self.disc(h1, h2, h3, h4, c1, c2)
        
        return out

In [ ]:
# 参数
hid_dim = 512
epochs = 50
patience = 20 # Patient epochs to wait before early stopping.
lr1 = 1e-3 # Learning rate of mvgrl
lr2 = 1e-2 # Learning rate of linear evaluator
wd1 = 5e-4
wd2 = 5e-4
device = 'cuda:0'
sample_size = 2000

In [ ]:
! nvidia-smi

Sat Mar 19 07:44:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P8    35W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# train

# 1. prepare data 
graph, diff_graph, feat, label, train_idx, val_idx, test_idx, edge_weight = process_dataset('cora')

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [ ]:
# 子图采样
graph.ndata['feat'] = feat
diff_graph.edata['edge_weight'] = edge_weight

n_feat = feat.shape[1]
n_classes = np.unique(label).shape[0]

train_idx = train_idx.to(device)
val_idx = val_idx.to(device)
test_idx = test_idx.to(device)

n_node = graph.number_of_nodes()

In [ ]:
# 2. create model, optimizer, loss function
model = MVGRL(n_feat, hid_dim)
model = model.to(device)

optimizer = th.optim.Adam(model.parameters(), lr=lr1, weight_decay=wd1)

loss_fn = nn.BCEWithLogitsLoss() 
# This loss combines a Sigmoid layer and the BCELoss in one single class.
# https://cloud.tencent.com/developer/article/1660961

In [ ]:
# 4. training epochs

lbl1 = th.ones(n_node * 2)
lbl2 = th.zeros(n_node * 2)
lbl = th.cat((lbl1, lbl2))
lbl = lbl.to(device)

# 把数据移动到device上
graph = graph.to(device)
diff_graph = diff_graph.to(device)
feat = feat.to(device)
edge_weight = edge_weight.to(device)

best = float('inf')
cnt_wait = 0 # early stopping

for epoch in range(epochs):
    model.train()
    
    # 打乱节点特征，相当于从边缘分布中采样
    shuf_idx = np.random.permutation(n_node)
    shuf_feat = feat[shuf_idx, :]
    shuf_feat = shuf_feat.to(device)

    logits = model(graph, diff_graph, feat, shuf_feat, edge_weight)
    loss = loss_fn(logits, lbl)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print('Epoch: {0}, Loss: {1:0.4f}'.format(epoch, loss.item()))
    
    if loss < best:
        best = loss
        cnt_wait = 0
        th.save(model.state_dict(), 'mvgrl.pkl') # 保存模型参数，不保存模型结构
    else: 
        cnt_wait += 1
        
    if cnt_wait == patience:
        print('Early stopping')
        break
    
model.load_state_dict(th.load('mvgrl.pkl'))
embeds = model.get_embedding(graph, diff_graph, feat, edge_weight)

/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


Epoch: 0, Loss: 0.6937
Epoch: 1, Loss: 0.6917


KeyboardInterrupt: ignored

In [ ]:
# sample 采样子图

lbl1 = th.ones(sample_size * 2)
lbl2 = th.zeros(sample_size * 2)
lbl = th.cat((lbl1, lbl2))
lbl = lbl.to(device)

node_list = list(range(n_node))

best = float('inf')
cnt_wait = 0 # early stopping

for epoch in range(epochs):
    model.train()

    sample_idx = random.sample(node_list, sample_size)

    sub_graph = dgl.node_subgraph(graph, sample_idx)
    sub_diff_graph = dgl.node_subgraph(diff_graph, sample_idx)

    f = sub_graph.ndata.pop('feat')
    ew = sub_diff_graph.edata.pop('edge_weight')

    shuf_idx = np.random.permutation(sample_size)
    sf = feat[shuf_idx, :]

    sub_graph = sub_graph.to(device)
    sub_diff_graph = sub_diff_graph.to(device)
    f = f.to(device)
    ew = ew.to(device)
    sf = sf.to(device)

    logits = model(sub_graph, sub_diff_graph, f, sf, ew)
    loss = loss_fn(logits, lbl)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print('Epoch: {0}, Loss: {1:0.4f}'.format(epoch, loss.item()))
    
    if loss < best:
        best = loss
        cnt_wait = 0
        th.save(model.state_dict(), 'mvgrl.pkl') # 保存模型参数，不保存模型结构
    else: 
        cnt_wait += 1
        
    if cnt_wait == patience:
        print('Early stopping')
        break

model.load_state_dict(th.load('mvgrl.pkl'))

# 把数据移动到device上
graph = graph.to(device)
diff_graph = diff_graph.to(device)
feat = feat.to(device)
edge_weight = edge_weight.to(device)
embeds = model.get_embedding(graph, diff_graph, feat, edge_weight)

Epoch: 0, Loss: 0.3746
Epoch: 1, Loss: 0.3621
Epoch: 2, Loss: 0.3546
Epoch: 3, Loss: 0.3579
Epoch: 4, Loss: 0.3452
Epoch: 5, Loss: 0.3453
Epoch: 6, Loss: 0.3404
Epoch: 7, Loss: 0.3267
Epoch: 8, Loss: 0.3364
Epoch: 9, Loss: 0.3303
Epoch: 10, Loss: 0.3197
Epoch: 11, Loss: 0.3282
Epoch: 12, Loss: 0.3228
Epoch: 13, Loss: 0.3088
Epoch: 14, Loss: 0.3155
Epoch: 15, Loss: 0.3010
Epoch: 16, Loss: 0.3232
Epoch: 17, Loss: 0.3130
Epoch: 18, Loss: 0.3072
Epoch: 19, Loss: 0.3006
Epoch: 20, Loss: 0.2960
Epoch: 21, Loss: 0.2883
Epoch: 22, Loss: 0.3067
Epoch: 23, Loss: 0.2866
Epoch: 24, Loss: 0.2895
Epoch: 25, Loss: 0.2943
Epoch: 26, Loss: 0.2912
Epoch: 27, Loss: 0.2943
Epoch: 28, Loss: 0.2913
Epoch: 29, Loss: 0.2853
Epoch: 30, Loss: 0.2751
Epoch: 31, Loss: 0.2794
Epoch: 32, Loss: 0.2922
Epoch: 33, Loss: 0.2769
Epoch: 34, Loss: 0.2701
Epoch: 35, Loss: 0.2800
Epoch: 36, Loss: 0.2867
Epoch: 37, Loss: 0.2643
Epoch: 38, Loss: 0.2783
Epoch: 39, Loss: 0.2694
Epoch: 40, Loss: 0.2741
Epoch: 41, Loss: 0.2684
Ep

In [ ]:
# down stream task
# node classification 只有节点的嵌入

# model
class LogReg(nn.Module):
    def __init__(self, hid_dim, n_classes):
        super(LogReg, self).__init__()
        
        self.fc = nn.Linear(hid_dim, n_classes)
        
    def forward(self, x):
        h = self.fc(x)
        # h = th.log_softmax(h, dim=-1)
        return h

# evaluation 
train_embs = embeds[train_idx]
test_embs = embeds[test_idx]

label = label.to(device)
train_labels = label[train_idx]
test_labels = label[test_idx]
accs = []

for _ in range(5):
  model = LogReg(hid_dim, n_classes)
  model = model.to(device)

  opt = th.optim.Adam(model.parameters(), lr=lr2, weight_decay=wd2)
  loss_fn = nn.CrossEntropyLoss()
  # https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html

  for epoch in range(300):
    model.train()

    logits = model(train_embs)
    loss = loss_fn(logits, train_labels) # target可以接受 shape (N) 或 (N, d_1, ..., d_k)

    opt.zero_grad()
    loss.backward()
    opt.step()

  model.eval()
  logits = model(test_embs)
  preds = th.argmax(logits, dim=1)
  acc = th.sum(preds == test_labels).float() / test_labels.shape[0]
  accs.append(acc * 100)

accs = th.stack(accs) # Concatenates a sequence of tensors along a new dimension. 类型转换
print(accs.mean().item(), accs.std().item())
  


79.80000305175781 0.0
